# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : Reproducibility
## Chapter : Riemannian metrics

In this notebook, we solve isotropic eikonal equations on the CPU and the GPU, and check that they produce consistent results.

In [1]:
import sys; sys.path.insert(0,"../..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Isotropic_Repro','GPU'))

In [2]:
import cupy as cp
import numpy as np
import itertools
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [3]:
from agd import HFMUtils
from agd import AutomaticDifferentiation as ad
from agd import Metrics
from agd import FiniteDifferences as fd
from agd import LinearParallel as lp
import agd.AutomaticDifferentiation.cupy_generic as cugen
norm_infinity = ad.Optimization.norm_infinity
from agd.HFMUtils import RunGPU,RunSmart

In [6]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global HFMUtils,ad,cugen,RunGPU,RunSmart,Metrics
    HFMUtils,ad,cugen,RunGPU,Metrics = rreload([HFMUtils,ad,cugen,RunGPU,Metrics],"../..")    
    RunSmart = cugen.cupy_get_args(HFMUtils.RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

In [5]:
cp = cugen.decorate_module_functions(cp,cugen.set_output_dtype32) # Use float32 and int32 types in place of float64 and int64
plt = cugen.decorate_module_functions(plt,cugen.cupy_get_args)
RunSmart = cugen.cupy_get_args(RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

### 0.1 Utilities

In [7]:
#from Notebooks_GPU.ExportedCode.Isotropic_Repro import RunCompare
def RunCompare(gpuIn,check=True):
    gpuOut = RunGPU(gpuIn)
    if gpuIn.get('verbosity',1): print("---")
    cpuIn = gpuIn.copy(); cpuIn.pop('traits',None)
    cpuOut = RunSmart(cpuIn)
    print("Max |gpuValues-cpuValues| : ", norm_infinity(gpuOut['values'].get()-cpuOut['values']))
    cpuTime = cpuOut['FMCPUTime']; gpuTime = gpuOut['solverGPUTime'];
    print(f"Solver time (s). GPU : {gpuTime}, CPU : {cpuTime}. Device acceleration : {cpuTime/gpuTime}")
    assert not check or cp.allclose(gpuOut['values'],cpuOut['values'],atol=1e-6)
    return gpuOut,cpuOut

In [8]:
factor_variants = [
    {}, # Default
    {"seedRadius":2}, # Spread seed information
    {"factorizationRadius":10,'factorizationPointChoice':'Key'} # Source factorization
]
multip_variants = [
    {'multiprecision':False}, # Default
    {'multiprecision':True}, # Reduces roundoff errors
]
order_variants = [
    {'order':1}, # Default
    {'order':2}, # More accurate on smooth instances
]

## 1. Two dimensions

### 1.1 Isotropic metric

In [9]:
n=4000
hfmIn = HFMUtils.dictIn({
    'model':'Riemann2',
    'metric':Metrics.Riemann(cp.eye(2)),
    'seeds':cp.array([[0.5,0.5]]),
    'exportValues':1,
    'traits':{
        'niter_i':24,'shape_i':(12,12), # Best
    }
})
hfmIn.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=True)

Casting output of function eye from float64 to float32
Casting output of function array from float64 to float32


In [9]:
_,cpuOut = RunCompare(hfmIn,check=False)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the values array (setting seeds,...)
Preparing the GPU kernel
Setup and run the eikonal solver
GPU solve took 0.4770023822784424 seconds, in 668 iterations.
Post-Processing
---
Field verbosity defaults to 1
Field order defaults to 1
Field seedRadius defaults to 0
Fast marching solver completed in 17.016 s.
Max |gpuValues-cpuValues| :  0.49105587362191727
Solver time (s). GPU : 0.4770023822784424, CPU : 17.016. Device acceleration : 35.672777814487276


In [10]:
n=200; hfmInS = hfmIn.copy() # Define a small instance for bit-consistency validation
hfmInS.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=True)
X = hfmInS.Grid()
cost = np.prod(np.sin(2*np.pi*X))+1.1
hfmInS.update({
    'metric': Metrics.Riemann(cost**2*fd.as_field(cp.eye(2),X.shape[1:])), # Isotropic but non-constant metric
    'verbosity':0,
})

Casting output of function eye from float64 to float32


In [11]:
RunCompare(hfmInS);

Max |gpuValues-cpuValues| :  4.017569794623199e-07
Solver time (s). GPU : 0.03299975395202637, CPU : 0.027. Device acceleration : 0.8181879185902854


In [12]:
for fact,multip in itertools.product(factor_variants,multip_variants):
    print(f"\nReproducibility with options : {fact}, {multip}")
    RunCompare({**hfmInS,**fact,**multip})


Reproducibility with options : {}, {'multiprecision': False}
Max |gpuValues-cpuValues| :  4.017569794623199e-07
Solver time (s). GPU : 0.008000373840332031, CPU : 0.027. Device acceleration : 3.3748422934795563

Reproducibility with options : {}, {'multiprecision': True}
Max |gpuValues-cpuValues| :  6.320754353250635e-08
Solver time (s). GPU : 0.010999679565429688, CPU : 0.027. Device acceleration : 2.4546169585573088

Reproducibility with options : {'seedRadius': 2}, {'multiprecision': False}
Max |gpuValues-cpuValues| :  4.202669309227858e-07
Solver time (s). GPU : 0.010000944137573242, CPU : 0.025. Device acceleration : 2.4997639878894797

Reproducibility with options : {'seedRadius': 2}, {'multiprecision': True}
Max |gpuValues-cpuValues| :  1.0889257084922832e-07
Solver time (s). GPU : 0.010991096496582031, CPU : 0.026. Device acceleration : 2.3655510629067242

Reproducibility with options : {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'}, {'multiprecision': False}
M

In [13]:
hfmInS.update({
    'seeds':[[0.0,1.]],
    'order':2,
})

In [15]:
for fact,multip in itertools.product((factor_variants[0],factor_variants[2]),multip_variants):
    print(f"\nReproducibility with options : {fact}, {multip}")
    RunCompare({**hfmInS,**fact,**multip})


Reproducibility with options : {}, {'multiprecision': False}
Max |gpuValues-cpuValues| :  4.88107854068609e-06
Solver time (s). GPU : 0.021503448486328125, CPU : 0.04. Device acceleration : 1.8601667553663297

Reproducibility with options : {}, {'multiprecision': True}
Max |gpuValues-cpuValues| :  1.291459792440719e-07
Solver time (s). GPU : 0.02249884605407715, CPU : 0.038. Device acceleration : 1.688975510506851

Reproducibility with options : {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'}, {'multiprecision': False}
Max |gpuValues-cpuValues| :  4.88107854068609e-06
Solver time (s). GPU : 0.02249908447265625, CPU : 0.041. Device acceleration : 1.8222963716514073

Reproducibility with options : {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'}, {'multiprecision': True}
Max |gpuValues-cpuValues| :  1.291459792440719e-07
Solver time (s). GPU : 0.022498607635498047, CPU : 0.04. Device acceleration : 1.7778877985715194


In [ ]:
### 1.2 